# DD2424 Deep Learning in Data Science - Lab 3

## Diar Sabri - July 2021

In [ ]:
# imports
import numpy as np
import pickle
import decimal
from google.colab import drive
drive.mount('/content/drive')
import copy
import matplotlib.pyplot as plt
import math
from scipy.linalg import fractional_matrix_power

In [ ]:
def LoadBatch(filename):
  """ Copied from the dataset website """
  with open('/content/drive/MyDrive/Colab Notebooks/cifar-10-batches-py/'+filename, 'rb') as fo:
    dict = pickle.load(fo, encoding='bytes')

  X = (dict[b'data'] / 255).T
  y = dict[b'labels']
  Y = (np.eye(10)[y]).T

  X = PreProcessData(X)

  return X,Y,y

In [ ]:
def LoadAllData():
  X = []
  Y = []
  y = []

  for i in range(1, 6):
    filename = 'data_batch_' + str(i)
    X_bat, Y_bat, y_bat = LoadBatch(filename)
    X.append(X_bat)
    Y.append(Y_bat)
    y.append(y_bat)
  
  return np.concatenate(X, axis=1), np.concatenate(Y, axis=1), np.concatenate(y, axis=0)

In [ ]:
def PreProcessData(X):
  mean_X = np.mean(X, axis=1).reshape(-1, 1)
  std_X = np.std(X, axis=1).reshape(-1, 1)

  X = (X - mean_X)/std_X

  return X

In [ ]:
def InitalizeLayers(hidden, d, K, alpha, vrs = []):
  layers = [item for sublist in [[d], hidden, [K]] for item in sublist]
  k = len(layers)-1
  NetParams = {}
  NetParams['alpha'] = alpha
  W = np.zeros(shape=(k,1)).tolist()
  b = np.zeros(shape=(k,1)).tolist()


  for i in range(0,k):
    std = np.sqrt(2 / layers[i])

    W[i] = np.random.normal(0, std, (layers[i+1],layers[i]))
    b[i] = np.zeros(shape=(layers[i+1],1))

  NetParams['gamma'] = np.zeros(shape=(len(hidden),1)).tolist()
  NetParams['beta'] = np.zeros(shape=(len(hidden),1)).tolist()

  for i in range(0,len(hidden)):
    NetParams['gamma'][i] = np.ones(shape=(hidden[i],1))#np.ones(hidden[i])
    NetParams['beta'][i] = np.zeros(shape=(hidden[i],1))#np.zeros(hidden[i])

  NetParams['W'] = W
  NetParams['b'] = b

  return NetParams

In [ ]:
def MiniBatchGD(tra_X, tra_Y, GDP, NetParams, lamb):

  training_loss, validation_loss = [], []
  
  D_size = len(tra_X[1])
  etas = CyclicalLRList(GDP['eta_min'], GDP['eta_max'], GDP['n_s'])

  for i in range(GDP['n_cycles']):
    GDP['c'] = GDP['c'] + 1
    for j in range(2*GDP['n_s']):
      j_start = (j * GDP['n_batch']) % D_size
      j_end = ((j + 1)* GDP['n_batch'] - 1) % D_size

      X_batch = tra_X[:, j_start:j_end+1]
      Y_batch = tra_Y[:, j_start:j_end+1]
      GDP['t'] = GDP['t'] + 1

      BNParams = ForwardPass(X_batch, NetParams)
      grad_W, grad_b = BackwardPass(X_batch, Y_batch, BNParams, NetParams, lamb)

      # Record the loss at the breaks between different epochs
      if j_start == 0 or j == 2*GDP['n_s'] - 1:
        global_step = i * 2 * GDP['n_s'] + j
        tra_loss = (global_step, ComputeCostVanilla(tra_X, tra_Y, NetParams, lamb))
        val_loss = (global_step, ComputeCostVanilla(GDP['val_X'], GDP['val_Y'], NetParams, lamb))

        training_loss.append(tra_loss)
        validation_loss.append(val_loss)

      # Update gradients
      cyclic_eta = etas[j]
      for q in range(len(NetParams['W'])):
        NetParams['W'][q] = NetParams['W'][q] - cyclic_eta * grad_W[q]
        NetParams['b'][q] = NetParams['b'][q] - cyclic_eta * grad_b[q]

  return NetParams, training_loss, validation_loss

In [ ]:
def ComputeCostVanilla(X, Y, NetParams, lamb):
  eval = ForwardPass(X, NetParams)
  tot_sum = 0
  for weight_matrix in NetParams['W']:
      tot_sum += np.sum(np.square(weight_matrix))

  loss = (1/X.shape[1]) * -np.sum(Y*np.log(eval['P']))

  j = loss + lamb * tot_sum

  return loss, j

In [ ]:
def ForwardPass(X, NetParams):
  W = NetParams['W']
  b = NetParams['b']

  k = len(W) - 1
  x = X
  hidden = []

  for i in range(k):
    s = W[i] @ x + b[i]
    s[s < 0] = 0
    x = copy.deepcopy(s)
    hidden.append(copy.deepcopy(s))

  x = W[-1] @ x + b[-1]
  P = np.exp(x) / np.sum(np.exp(x), axis=0)

  BNParams = {}
  BNParams['P'] = P
  BNParams['hidden'] = hidden

  return BNParams

In [ ]:
def BackwardPass(X, Y, BNParams, NetParams ,lam):
  W = NetParams['W']
  b = NetParams['b']
  P = BNParams['P']
  hidden = BNParams['hidden']

  G = -(Y - P)

  k = len(W) - 1
  grad_W = []
  grad_b = []

  for l in range(k, 0, -1):
    grad_W.append(G @ hidden[l-1].T / X.shape[1] + 2 * lam * W[l])
    grad_b.append(np.mean(G, axis=-1, keepdims=True))

    G = W[l].T @ G
    G = np.multiply(G, hidden[l-1] > 0)

  w = G @ X.T / X.shape[1] + 2 * lam * W[0]
  b = np.mean(G, axis=-1, keepdims=True)

  grad_W.append(w)
  grad_b.append(b)

  grad_W.reverse()
  grad_b.reverse()

  return grad_W, grad_b

In [ ]:
def MiniBatchGDBN(tra_X, tra_Y, tra_y, GDP, NetParams, lamb):

  # Init the means and variances before the main loop
  BNParams = ForwardPassBN(tra_X[:, :100], NetParams)
  my_avg = BNParams['my']
  v_avg = BNParams['v']

  training_loss, validation_loss = [], []
  
  D_size = len(tra_X[1])
  etas = CyclicalLRList(GDP['eta_min'], GDP['eta_max'], GDP['n_s'])

  for i in range(GDP['n_cycles']):
    GDP['c'] = GDP['c'] + 1
    for j in range(2*GDP['n_s']):
      j_start = (j * GDP['n_batch']) % D_size
      j_end = ((j + 1)* GDP['n_batch'] - 1) % D_size

      X_batch = tra_X[:, j_start:j_end+1]
      Y_batch = tra_Y[:, j_start:j_end+1]
      GDP['t'] = GDP['t'] + 1

      BNParams = ForwardPassBN(X_batch, NetParams)
      grad_W, grad_b, grad_beta, grad_gamma = BackwardPassBN(X_batch, Y_batch, BNParams, NetParams, lamb)

      # Record the loss at the breaks between different epochs
      if j_start == 0 or j == 2*GDP['n_s'] - 1:
        global_step = i * 2 * GDP['n_s'] + j

        tra_loss = ComputeCostWAcc(tra_X, tra_Y, tra_y, NetParams, my_avg, v_avg, lamb)
        val_loss = ComputeCostWAcc(GDP['val_X'], GDP['val_Y'], GDP['val_y'], NetParams, my_avg, v_avg, lamb)
        
        tra_loss = (global_step, (tra_loss[0], tra_loss[1]))
        val_loss = (global_step, (val_loss[0], val_loss[1]))

        training_loss.append(tra_loss)
        validation_loss.append(val_loss)

      # Update avg means and variances
      for k in range(len(my_avg)):
        my_avg[k] = NetParams['alpha'] * my_avg[k] + (1 - NetParams['alpha']) * BNParams['my'][k]
        v_avg[k] = NetParams['alpha'] * v_avg[k] + (1 - NetParams['alpha']) * BNParams['v'][k]

      # Update weight matrixes
      cyclic_eta = etas[j]
      for q in range(len(NetParams['W'])):
        NetParams['W'][q] = NetParams['W'][q] - cyclic_eta * grad_W[q]
        NetParams['b'][q] = NetParams['b'][q] - cyclic_eta * grad_b[q]
        if q < len(grad_W) - 1:
          NetParams['beta'][q] = NetParams['beta'][q] - cyclic_eta * grad_beta[q]
          NetParams['gamma'][q] = NetParams['gamma'][q] - cyclic_eta * grad_gamma[q]
          
  return NetParams, training_loss, validation_loss

In [ ]:
def CyclicalLRList(eta_min, eta_max, n_s):
  diff = eta_max - eta_min
  etas = []
  for t in range(2 * n_s):
    val = eta_min + t/n_s * diff if t <= n_s else eta_max - (t - n_s)/n_s * diff
    etas.append(val)
  
  return etas

In [ ]:
def ForwardPassBN(X, NetParams, MU=None, V=None):
  if MU is None:
    MU = []
  if V is None:
    V = []
  
  W = NetParams['W']
  b = NetParams['b']
  beta = NetParams['beta']
  gamma = NetParams['gamma']
  
  test_time = len(MU) > 0 and len(V) > 0
  k = len(W) - 1

  S = []
  S_hat = []
  x = [X]
  for l in range(k):
    s = W[l] @ x[l] + b[l]
    S.append(s)

    if test_time:
      mu = MU[l]
      v = V[l]
    else:
      mu = np.mean(s, axis=1, keepdims=True)
      MU.append(mu)
      v = np.var(s, axis=1, keepdims=True)
      V.append(v)

    s_hat = BatchNormalize(s, mu, v)
    S_hat.append(s_hat)

    s_scaled = np.multiply(gamma[l], s_hat) + beta[l]
    
    s_scaled[s_scaled < 0] = 0
    x.append(s_scaled)

  s = W[-1] @ x[-1] + b[-1]
  P = np.exp(x) / np.sum(np.exp(x), axis=0)

  BNParams = {}
  BNParams['s'] = S
  BNParams['s_h'] = S_hat
  BNParams['x'] = x
  BNParams['my'] = MU
  BNParams['v'] = V
  BNParams['P'] = P

  return BNParams

In [ ]:
def BatchNormalize(s, mu, v):
  v_flat_eps = v.flatten() + np.finfo(float).eps
  return fractional_matrix_power(np.diag(v_flat_eps), -0.5) @ (s - mu)

In [ ]:
def BackwardPassBN(X, Y, BNParams, NetParams ,lam):
  W = NetParams['W']
  b = NetParams['b']
  gamma = NetParams['gamma']
  beta = NetParams['beta']

  S = BNParams['s']
  S_hat = BNParams['s_h']
  X_batch = BNParams['x']
  MU = BNParams['my']
  V = BNParams['v']
  P = BNParams['P']

  n = X.shape[1]
  ones = np.ones(n).reshape(-1, 1)
  
  G = -(Y - P)
  w_k_grad = 1 / n * G @ X_batch[-1].T + 2 * lam * W[-1]
  b_k_grad = 1 / n * G @ ones

  G = W[-1].T @ G
  G = np.multiply(G, X_batch[-1] > 0)

  k = len(W) - 1
  grad_W = [w_k_grad]
  grad_b = [b_k_grad]
  grad_beta = []
  grad_gamma = []
  for l in range(k-1, -1, -1):
    grad_beta.append(1 / n * G @ ones)
    grad_gamma.append(1 / n * np.multiply(G, S_hat[l]) @ ones)

    G = np.multiply(G, np.reshape(gamma[l],(-1,1)) @ ones.T)
    G = BNBackPass(G, S[l], MU[l], V[l])

    grad_W.append(1 / n * G @ X_batch[l].T + 2 * lam * W[l])
    grad_b.append(1 / n * G @ ones)

    if l > 0:
      G = W[l].T @ G
      G = np.multiply(G, X_batch[l] > 0)

  grad_W.reverse()
  grad_b.reverse()
  grad_beta.reverse()
  grad_gamma.reverse()

  return grad_W, grad_b, grad_beta, grad_gamma

In [ ]:
def BNBackPass(G, S, mu, v):
  n = S.shape[1]
  eps = np.finfo(float).eps
  ones = np.ones(n).reshape(-1, 1)
  sigma_one = np.array([pow((v_i + eps), -0.5) for v_i in v])
  sigma_two = np.array([pow((v_i + eps), -1.5) for v_i in v])
  
  G1 = np.multiply(G, sigma_one @ ones.T)
  G2 = np.multiply(G, sigma_two @ ones.T)
  D = S - mu @ ones.T
  c = np.multiply(G2, D) @ ones

  return G1 - 1 / n * (G1 @ ones) @ ones.T - 1/n * np.multiply(D, c @ ones.T)  

In [ ]:
def ComputeCostWAcc(X, Y, y, NetParams, my_avg, v_avg, lamb):
  eval = ForwardPassBN(X, NetParams, [my_avg, v_avg])
  tot_sum = 0
  for weight_matrix in NetParams['W']:
      tot_sum += np.sum(np.square(weight_matrix))

  loss = (1/X.shape[1]) * -np.sum(Y*np.log(eval['P']))
  j = loss + lamb * tot_sum

  acc = ComputeAcc(y, eval['P'])
  return loss, j, acc

In [ ]:
def ComputeCost(X, Y, NetParams, lamb):
  eval = ForwardPassBN(X, NetParams)
  tot_sum = 0
  for weight_matrix in NetParams['W']:
      tot_sum += np.sum(np.square(weight_matrix))

  loss = (1/X.shape[1]) * -np.sum(Y*np.log(eval['P']))
  j = loss + lamb * tot_sum

  return loss, j

In [ ]:
def ComputeAcc(y, P):
  preds = [np.argmax(pred) for pred in P.T]
  corr = sum([1 for i in range(len(y)) if preds[i] == y[i]])
  acc = corr/len(y) 

  return acc

In [ ]:
def PlotLoss(training_loss, validation_loss):

  tra_X = [x for x, (_, y) in training_loss]
  tra_Y = [y for x, (_, y) in training_loss]

  val_X = [x for x, (_, y) in validation_loss]
  val_Y = [y for x, (_, y) in validation_loss]

  plt.plot(tra_X, tra_Y, label='Traning')
  plt.plot(val_X, val_Y, label='Validation')
  plt.legend()

  plt.show()

# Gradients

In [ ]:
def ComputeGradsNumSlowBN(X, Y, BNParams, NetParams, lam, h):
  grad_W = []
  grad_b = []
  grad_beta = []
  grad_gamma = []

  for l in range(len(NetParams['W'])):
    grad_W.append(np.zeros(NetParams['W'][l].shape))
    grad_b.append(np.zeros(NetParams['b'][l].shape))

    for i in range(len(NetParams['b'][l])):
      NetParams_try = copy.deepcopy(NetParams)
      NetParams_try['b'][l][i] -= h
      _, c1 = ComputeCost(X, Y, NetParams_try, lam)

      NetParams_try = copy.deepcopy(NetParams)
      NetParams_try['b'][l][i] += h
      _, c2 = ComputeCost(X, Y, NetParams_try, lam)

      grad_b[l][i] = (c2 - c1) / (2 * h)

    for i in range(NetParams['W'][l].shape[0]):
      for j in range(NetParams['W'][l].shape[1]):
        NetParams_try = copy.deepcopy(NetParams)
        NetParams_try['W'][l][i, j] -= h
        _, c1 = ComputeCost(X, Y, NetParams_try, lam)

        NetParams_try = copy.deepcopy(NetParams)
        NetParams_try['W'][l][i, j] += h
        _, c2 = ComputeCost(X, Y, NetParams_try, lam)
        
        grad_W[l][i,j] = (c2 - c1) / (2 * h)
    
    if l < len(NetParams['W']) - 1:
      grad_beta.append(np.zeros((len(NetParams['beta'][l]), 1)))
      grad_gamma.append(np.zeros((len(NetParams['gamma'][l]), 1)))

      for i in range(len(NetParams['beta'][l])):
        NetParams_try = copy.deepcopy(NetParams)
        NetParams_try['beta'][l][i] -= h
        _, c1 = ComputeCost(X, Y, NetParams_try, lam)

        NetParams_try = copy.deepcopy(NetParams)
        NetParams_try['beta'][l][i] += h
        _, c2 = ComputeCost(X, Y, NetParams_try, lam)

        grad_beta[l][i] = (c2 - c1) / (2 * h)
      
      for i in range(len(NetParams['gamma'][l])):
        NetParams_try = copy.deepcopy(NetParams)
        NetParams_try['gamma'][l][i] -= h
        _, c1 = ComputeCost(X, Y, NetParams_try, lam)

        NetParams_try = copy.deepcopy(NetParams)
        NetParams_try['gamma'][l][i] += h
        _, c2 = ComputeCost(X, Y, NetParams_try, lam)

        grad_gamma[l][i] = (c2 - c1) / (2 * h)
    
  return grad_W, grad_b, grad_beta, grad_gamma

In [ ]:
def AbsDiff(analytical, numerical, tolerance=1e-05):
  assert analytical.shape == numerical.shape
  analytical = analytical.flatten()
  numerical = numerical.flatten()
  equal = True
  for i in range(len(analytical)):
    diff = abs(analytical[i] - numerical[i])
    if diff >= tolerance:
      print(analytical[i], numerical[i], diff)
      equal = False
  return equal

In [ ]:
def CompareGradientsBN():
  samples = 10
  hidden = [50, 50]
  d = 10
  K = 10
  layers = len(hidden)
  alpha = 0.9
  lam = 0
  h = 1e-5
  tolerance = 1e-5

  X, Y, _ = LoadBatch('data_batch_1')
  X = X[:d, :samples]
  Y = Y[:, :samples]
  
  NetParams = InitalizeLayers(hidden, d, K, alpha)
  BNParams = ForwardPassBN(X, NetParams)

  grad_W, grad_b, grad_beta, grad_gamma = BackwardPassBN(X, Y, BNParams, NetParams, lam)
  grad_W_num, grad_b_num, grad_beta_num, grad_gamma_num = ComputeGradsNumSlowBN(X, Y, BNParams, NetParams, lam, h)


  layer_equal = []
  for l in range(len(grad_W)):
    W_equal = AbsDiff(grad_W[l], grad_W_num[l], tolerance)
    b_equal = AbsDiff(grad_b[l], grad_b_num[l], tolerance)
    result = {
      'W': W_equal,   
      'b': b_equal,   
    }

    if l < len(grad_W) - 1:
      beta_equal = AbsDiff(grad_beta[l], grad_beta_num[l], tolerance)
      gamma_equal = AbsDiff(grad_gamma[l], grad_gamma_num[l], tolerance)
      result['beta'] = beta_equal
      result['gamma'] = gamma_equal

    else:
      result['beta'] = None
      result['gamma'] = None
    
    layer_equal.append(result)
    
  grads = {}
  grads['W'] = grad_W
  grads['b'] = grad_b
  grads['beta'] = grad_beta
  grads['gamma'] = grad_gamma

  grads_num = {}
  grads_num['W'] = grad_W_num
  grads_num['b'] = grad_b_num
  grads_num['beta'] = grad_beta_num
  grads_num['gamma'] = grad_gamma_num

  return grads, grads_num, layer_equal

In [ ]:
grads, grads_num, layers_compared = CompareGradientsBN()
layers_compared

# Batch-norm vs Normal

## Load Data

In [ ]:
test_X, test_Y, test_y = LoadBatch('test_batch')
all_X, all_Y, all_y = LoadAllData()

In [ ]:
def PartData(validation_size):
  val_X = all_X[:, :validation_size]
  val_Y = all_Y[:, :validation_size]
  val_y = all_y[:validation_size]

  tra_X = all_X[:, validation_size:]
  tra_Y = all_Y[:, validation_size:]
  tra_y = all_y[validation_size:]

  return val_X, val_Y, val_y, tra_X, tra_Y, tra_y

## 3 Layers

In [ ]:
#3 Layer experiment with no bn
def Exp3LNoBN():
  print('3 Layers - NO BATCH NORM')

  validation_size = 5000
  val_X, val_Y, val_y, tra_X, tra_Y, tra_y = PartData(validation_size)

  d = len(tra_X)
  K = len(tra_Y)

  # The shape of the network 
  hidden = [50, 50]
  alpha = 0.9
  NetParams = InitalizeLayers(hidden, d, K, alpha)

  # Hyper-paramters for the experiment
  GDP = {}
  GDP['n_cycles'] = 2
  GDP['n_batch'] = 100
  GDP['n_s'] = int(np.floor(5*45000 / GDP['n_batch']))
  GDP['t'] = int(0)
  GDP['c'] = int(0)
  GDP['eta_min'] = 1e-5
  GDP['eta_max'] = 1e-1
  GDP['val_X'] = val_X
  GDP['val_Y'] = val_Y
  GDP['val_y'] = val_y

  lamb = 0.005

  NetParams, training_loss, validation_loss = MiniBatchGD(tra_X, tra_Y, GDP, NetParams, lamb)


  P_test = ForwardPass(test_X, NetParams)['P']
  acc_test = ComputeAcc(test_y, P_test)
  print('Test accuracy: ' + str(acc_test))

  PlotLoss(training_loss, validation_loss)

  return NetParams, training_loss, validation_loss

final_model, final_training_loss, final_validation_loss = Exp3LNoBN()

In [ ]:
#3 Layer experiment with bn
def Exp3LBN():
  print('3 Layers - BATCH NORM')

  validation_size = 5000
  val_X, val_Y, val_y, tra_X, tra_Y, tra_y = PartData(validation_size)

  d = len(tra_X)
  K = len(tra_Y)

  # The shape of the network 
  hidden = [50, 50]
  alpha = 0.9
  NetParams = InitalizeLayers(hidden, d, K, alpha)

  # Hyper-paramters for the experiment
  GDP = {}
  GDP['n_cycles'] = 2
  GDP['n_batch'] = 100
  GDP['n_s'] = int(np.floor(5*45000 / GDP['n_batch']))
  GDP['t'] = int(0)
  GDP['c'] = int(0)
  GDP['eta_min'] = 1e-5
  GDP['eta_max'] = 1e-1
  GDP['val_X'] = val_X
  GDP['val_Y'] = val_Y
  GDP['val_y'] = val_y

  lamb = 0.005

  NetParams, training_loss, validation_loss = MiniBatchGDBN(tra_X, tra_Y, tra_y, GDP, NetParams, lamb)

  P_test = ForwardPassBN(test_X, NetParams)['P']
  acc_test = ComputeAcc(test_y, P_test)
  print('Test accuracy: ' + str(acc_test))

  PlotLoss(training_loss, validation_loss)

  return NetParams, training_loss, validation_loss

final_model, final_training_loss, final_validation_loss = Exp3LBN()

## 9 Layers

In [ ]:
#9 Layer experiment with no bn
def Exp9LNoBN():
  print('9 Layers - NO BATCH NORM')

  validation_size = 5000
  val_X, val_Y, val_y, tra_X, tra_Y, tra_y = PartData(validation_size)

  d = len(tra_X)
  K = len(tra_Y)

  # The shape of the network 
  hidden = [50, 30, 20, 20, 10, 10, 10, 10]
  alpha = 0.9
  NetParams = InitalizeLayers(hidden, d, K, alpha)

  # Hyper-paramters for the experiment
  GDP = {}
  GDP['n_cycles'] = 2
  GDP['n_batch'] = 100
  GDP['n_s'] = int(np.floor(5*45000 / GDP['n_batch']))
  GDP['t'] = int(0)
  GDP['c'] = int(0)
  GDP['eta_min'] = 1e-5
  GDP['eta_max'] = 1e-1
  GDP['val_X'] = val_X
  GDP['val_Y'] = val_Y
  GDP['val_y'] = val_y

  lamb = 0.005

  NetParams, training_loss, validation_loss = MiniBatchGD(tra_X, tra_Y, GDP, NetParams, lamb)

  P_test = ForwardPass(test_X, NetParams)['P']
  acc_test = ComputeAcc(test_y, P_test)
  print('Test accuracy: ' + str(acc_test))

  PlotLoss(training_loss, validation_loss)

  return NetParams, training_loss, validation_loss

final_model, final_training_loss, final_validation_loss = Exp9LNoBN()

In [ ]:
#9 Layer experiment with bn
def Exp9LBN():
  print('9 Layers - BATCH NORM')

  validation_size = 5000
  val_X, val_Y, val_y, tra_X, tra_Y, tra_y = PartData(validation_size)

  d = len(tra_X)
  K = len(tra_Y)

  # The shape of the network 
  hidden = [50, 30, 20, 20, 10, 10, 10, 10]
  alpha = 0.9
  NetParams = InitalizeLayers(hidden, d, K, alpha)

  # Hyper-paramters for the experiment
  GDP = {}
  GDP['n_cycles'] = 2
  GDP['n_batch'] = 100
  GDP['n_s'] = int(np.floor(5*45000 / GDP['n_batch']))
  GDP['t'] = int(0)
  GDP['c'] = int(0)
  GDP['eta_min'] = 1e-5
  GDP['eta_max'] = 1e-1
  GDP['val_X'] = val_X
  GDP['val_Y'] = val_Y
  GDP['val_y'] = val_y

  lamb = 0.005

  NetParams, training_loss, validation_loss = MiniBatchGDBN(tra_X, tra_Y, tra_y, GDP, NetParams, lamb)

  P_test = ForwardPassBN(test_X, NetParams)['P']
  acc_test = ComputeAcc(test_y, P_test)
  print('Test accuracy: ' + str(acc_test))

  PlotLoss(training_loss, validation_loss)

  return NetParams, training_loss, validation_loss

final_model, final_training_loss, final_validation_loss = Exp9LBN()

# Lambda

In [ ]:
# Lambda Optimization
def ExpLambdaOpt():
  print('Lambda Optimization')

  validation_size = 5000
  val_X, val_Y, val_y, tra_X, tra_Y, tra_y = PartData(validation_size)

  d = len(tra_X)
  K = len(tra_Y)

  # The shape of the network 
  hidden = [50, 50]
  alpha = 0.9

  # Hyper-paramters for the experiment
  GDP = {}
  GDP['n_cycles'] = 2
  GDP['n_batch'] = 100
  GDP['n_s'] = int(np.floor(5*45000 / GDP['n_batch']))
  GDP['t'] = int(0)
  GDP['c'] = int(0)
  GDP['eta_min'] = 1e-5
  GDP['eta_max'] = 1e-1
  GDP['val_X'] = val_X
  GDP['val_Y'] = val_Y
  GDP['val_y'] = val_y

  n_lambdas = 10
  l_min = -5
  l_max = -1
  grid = np.linspace(0, 1, n_lambdas)
  lambdas = [np.power(10, l_min + (l_max - l_min) * val) for val in grid]
  
  for lamb in lambdas:
    NetParams = InitalizeLayers(hidden, d, K, alpha)
    NetParams, _, _ = MiniBatchGDBN(tra_X, tra_Y, tra_y, GDP, NetParams, lamb)

    P_test = ForwardPassBN(test_X, NetParams)['P']
    acc_test = ComputeAcc(test_y, P_test)
    print(str(lamb) + ' - Test accuracy: ' + str(acc_test))

ExpLambdaOpt()

# Sensitivity to initialization

In [ ]:
def InitLFixedSigma(hidden, d, K, alpha, sigma, vrs = []):
  layers = [item for sublist in [[d], hidden, [K]] for item in sublist]
  k = len(layers)-1
  NetParams = {}
  NetParams['alpha'] = alpha
  W = np.zeros(shape=(k,1)).tolist()
  b = np.zeros(shape=(k,1)).tolist()


  for i in range(0,k):
    ##if not vrs:
      ##std = np.sqrt(2/layers[i])
    ##else:
      ##std = vrs[0]

    std = sigma

    W[i] = np.random.normal(0, std, (layers[i+1],layers[i]))
    b[i] = np.zeros(shape=(layers[i+1],1))

  NetParams['gamma'] = np.zeros(shape=(len(hidden),1)).tolist()
  NetParams['beta'] = np.zeros(shape=(len(hidden),1)).tolist()

  for i in range(0,len(hidden)):
    NetParams['gamma'][i] = np.ones(shape=(hidden[i],1))#np.ones(hidden[i])
    NetParams['beta'][i] = np.zeros(shape=(hidden[i],1))#np.zeros(hidden[i])

  NetParams['W'] = W
  NetParams['b'] = b

  return NetParams

In [ ]:
# Fixed sigma experiment with no bn
def ExpFixedSigma():
  print('Fixed Sigma - NO BN')

  validation_size = 5000
  val_X, val_Y, val_y, tra_X, tra_Y, tra_y = PartData(validation_size)

  d = len(tra_X)
  K = len(tra_Y)

  # The shape of the network 
  hidden = [50, 50]
  alpha = 0.9
  lamb = 0.005

  # Hyper-paramters for the experiment
  GDP = {}
  GDP['n_cycles'] = 2
  GDP['n_batch'] = 100
  GDP['n_s'] = int(np.floor(5*45000 / GDP['n_batch']))
  GDP['t'] = int(0)
  GDP['c'] = int(0)
  GDP['eta_min'] = 1e-5
  GDP['eta_max'] = 1e-1
  GDP['val_X'] = val_X
  GDP['val_Y'] = val_Y
  GDP['val_y'] = val_y

  sigmas = [1e-1, 1e-3, 1e-4]

  for sigma in sigmas:
    NetParams = InitLFixedSigma(hidden, d, K, alpha, sigma)
    NetParams, training_loss, validation_loss = MiniBatchGD(tra_X, tra_Y, GDP, NetParams, lamb)

    P_test = ForwardPassBN(test_X, NetParams)['P']
    acc_test = ComputeAcc(test_y, P_test)
    print(str(sigma) + ' - Test accuracy: ' + str(acc_test))

    PlotLoss(training_loss, validation_loss)

ExpFixedSigma()

In [ ]:
# Fixed sigma experiment with bn
def ExpFixedSigmaBN():
  print('Fixed Sigma - BN')

  validation_size = 5000
  val_X, val_Y, val_y, tra_X, tra_Y, tra_y = PartData(validation_size)

  d = len(tra_X)
  K = len(tra_Y)

  # The shape of the network 
  hidden = [50, 50]
  alpha = 0.9
  lamb = 0.005

  # Hyper-paramters for the experiment
  GDP = {}
  GDP['n_cycles'] = 2
  GDP['n_batch'] = 100
  GDP['n_s'] = int(np.floor(5*45000 / GDP['n_batch']))
  GDP['t'] = int(0)
  GDP['c'] = int(0)
  GDP['eta_min'] = 1e-5
  GDP['eta_max'] = 1e-1
  GDP['val_X'] = val_X
  GDP['val_Y'] = val_Y
  GDP['val_y'] = val_y

  sigmas = [1e-1, 1e-3, 1e-4]

  for sigma in sigmas:
    NetParams = InitLFixedSigma(hidden, d, K, alpha, sigma)
    NetParams, training_loss, validation_loss = MiniBatchGDBN(tra_X, tra_Y, tra_y, GDP, NetParams, lamb)

    P_test = ForwardPassBN(test_X, NetParams)['P']
    acc_test = ComputeAcc(test_y, P_test)
    print(str(sigma) + ' - Test accuracy: ' + str(acc_test))

    PlotLoss(training_loss, validation_loss)

ExpFixedSigmaBN()